# Social and Informational Influences

This section explores how social context and information sources shape the adoption of electric vehicles (EVs) in France. Survey data was analyzed to understand the relationships between:

- **Peer influence**  
  (Number of EV users in the respondent’s social circle, discussions about EVs, encouragement or discouragement from peers, Perceived opinion of peers, Impact of peer exchanges)
- **Clarity and trust in information sources**  
  (Perceived clarity and reliability of information about EVs, Main information sources)
- **Brand communication**  
  (Brands perceived as offering clear and well-communicated EV options)
- **Sociodemographic effects**  
  (Influence of age, professional status, and geographic location on adoption intent)

## Setup

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from scipy.stats import chi2_contingency
import re
from scipy.stats import kruskal
import pandas as pd
from scipy.stats import f_oneway
from scipy.stats import spearmanr
from IPython.display import display
from scipy.stats import pearsonr
from wordcloud import WordCloud, STOPWORDS
from scipy.stats import ttest_ind, f_oneway

In [17]:
def extract_after_newline_parentheses(col):
    match = re.search(r'\n\s*\((.*?)\)', col)
    if match:
        return match.group(1)
    match = re.search(r'\(([^()]*(?:\([^)]*\)[^()]*)*)\)\s*$', col)
    if match:
        return match.group(1)
    return None

def extract_before_newline_parentheses(col):
    match = re.search(r'^(.*?)\s*\n', col)
    if match:
        return match.group(1)
    match = re.search(r'^(.*?)\s*\(', col)
    if match:
        return match.group(1)
    return col
def get_labels_and_title(cols):
    labels = [extract_after_newline_parentheses(col) for col in cols]
    title = extract_before_newline_parentheses(cols[0])
    return labels, title



In [18]:
df = pd.read_csv(r'FinalEvAdoptionInFranceResults.csv')
# for col in df.columns:
#     values = df[col].dropna().astype(str)
#     split_vals = []
#     for val in values:
#         split_vals.extend([v.strip() for v in val.split(',') if v.strip()])
#     unique_vals = pd.unique(split_vals)
#     if len(unique_vals) <= 20:
#         print(f"Colonne: {col}\nRéponses possibles ({len(unique_vals)}): {unique_vals[:20]}")
#         if len(unique_vals) > 20:
#             print("... (troncature à 20 valeurs)")
#         print("-" * 60)
# print(list(df.columns))

In [19]:
# Création des colonnes de base pour l'analyse


df['owns_ev'] = df["Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Électrique)"].fillna(False).astype(bool)

# Région (origine géographique)
region_col = [col for col in df.columns if "Habitez-vous dans" in col][0]
df['region'] = df[region_col]

# Tranche d'âge
age_col = [col for col in df.columns if "tranche d’âge" in col][0]
df['age_group'] = df[age_col]

# Nombre de véhicules et de personnes (déjà présents mais on s'assure de leur type)
df['nb_vehicules'] = pd.to_numeric(df['Combien de véhicules personnels utilisez-vous dans votre foyer ?'], errors='coerce')
df['nb_personnes'] = pd.to_numeric(df['Combien êtes-vous dans votre foyer ?'], errors='coerce')

vehicle_cols = [
    'Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Essence ou Diesel)',
    'Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Hybride non rechargeable)',
    'Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Hybride rechargeable)',
    'Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Électrique)',
]
driving_exp_col = "Dans quelle mesure êtes-vous d’accord avec les affirmations suivantes concernant les véhicules électriques ?\n [Un véhicule électrique correspondrait à mon mode de vie actuel]"

# Recodage échelle 4 points
likert_map = {
    "Totalement en désaccord": 1,
    "Plutôt en désaccord": 2,
    "Plutôt d'accord": 3,
    "Tout à fait d'accord": 4,
    "Tout à fait d’accord": 4,  # gestion des accents différents
}
df['intention_adopter'] = df["Diriez-vous que : [Ne possédant pas de véhicule 100% électrique, vous vous sentez attiré(e) par cette technologie et envisagez de l'adopter]"].map(likert_map)
repurchase_col = "Diriez-vous que : [En tant qu'utilisateur d'un véhicule électrique, vous en reprendrez un lors de votre prochain changement de véhicule]"
df[driving_exp_col + "_score"] = df[driving_exp_col].map(likert_map)
df['repurchase_score'] = df[repurchase_col].map(likert_map)


exp_col = "Avez-vous déjà eu l’occasion de conduire ou de monter dans un véhicule électrique ?"
connaissance_col = "À quel point diriez-vous que vous connaissez bien les offres actuelles de véhicules électriques et d'infrastructures associées ?"

# Recodage expérience : 1 = Oui, 0 = Non
df['exp_ev'] = df[exp_col].map({'Oui, j’ai conduit': 3,'Oui, j’étais passager': 2,'Non, jamais': 1})


connaissance_map = {
        "Pas du tout, je ne m’y suis jamais vraiment intéressé": 1,
        "Un peu, je connais quelques modèles et options": 2,
        "Assez bien, je vois passer des informations": 3,
        "Très bien, je suis régulièrement les nouveautés": 4
    }
df['connaissance_score'] = df[connaissance_col].map(connaissance_map)

aides_col = "Connaissez-vous des aides financières disponibles pour l’achat d’un véhicule électrique ? (Cochez tout ce que vous connaissez)\n"
def count_aides(cell):
    if pd.isna(cell) or not cell.strip():
        return 0
    return len([aide.strip() for aide in cell.split(',') if aide.strip() and aide.strip().lower() != "aucune"])

df['score_aides_connues'] = df[aides_col].apply(count_aides)
stopwords = set(STOPWORDS)


C:\Users\Mango\AppData\Local\Temp\ipykernel_28968\2005884872.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['owns_ev'] = df["Quels types de véhicules possédez vous dans votre foyer (plusieurs réponses possibles) \n (Électrique)"].fillna(False).astype(bool)


## **Peer influence**  
  (Number of EV users in the respondent’s social circle, discussions about EVs, encouragement or discouragement from peers, Perceived opinion of peers, Impact of peer exchanges)

In [20]:
peer_cols = [
    "Combien de personnes dans votre entourage utilisent un véhicule électrique ?\n",
    "Vous diriez que [Vous avez déjà échangé sur le sujet des véhicules électriques avec votre entourage]",
    "Vous diriez que [l'avis de votre entourage vous à encourager dans l'adoption du véhicule électrique]",
    "Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique]",
    "Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet]"
]
for col in peer_cols:
    print(f"\n{col}")
    print(df[col].value_counts(dropna=False))
    if df[col].dtype == object:
        # Recoder Oui/Non si besoin
        df[col + "_bin"] = df[col].map({"Oui": 1, "Non": 0, "Vaguement": 0.5, "Je ne sais pas": np.nan})
        group = df.groupby(col + "_bin")["intention_adopter"].mean()
        print("Intention d'adopter selon ce facteur :\n", group)

        for col in peer_cols[1:]:  # Sauf la première qui est quantitative
            group1 = df[df[col + "_bin"] == 1]["intention_adopter"].dropna()
            group0 = df[df[col + "_bin"] == 0]["intention_adopter"].dropna()
            print(f"{col} - n1: {len(group1)}, n0: {len(group0)}")
            if len(group1) > 1 and len(group0) > 1:
                tstat, pval = ttest_ind(group1, group0, equal_var=False)
                print(f"{col} : t = {tstat:.2f}, p = {pval:.4f}")
                if pval < 0.05:
                    print("✅ Différence significative entre les groupes.")
                else:
                    print("❌ Pas de différence significative entre les groupes.")
            else:
                print("Pas assez de données pour le t-test.")

try:
    # Recodage des modalités textuelles en valeurs numériques approximatives
    mapping_nb = {
        "Aucune": 0,
        "Peu (1 à 3)": 2,
        "Plusieurs (3 à 5)": 4,
        "Beaucoup (plus de 5)": 6
    }
    entourage_nb = df[peer_cols[0]].map(mapping_nb)

    corr = entourage_nb.corr(df['intention_adopter'])
    print(f"Corrélation entre nombre d'utilisateurs VE dans l'entourage et intention d'adopter : r = {corr:.2f}")
except Exception as e:
    print("Erreur de calcul de corrélation :", e)



Combien de personnes dans votre entourage utilisent un véhicule électrique ?

Combien de personnes dans votre entourage utilisent un véhicule électrique ?\n
Peu (1 à 3)             121
Aucune                   64
Plusieurs (3 à 5)        30
Beaucoup (plus de 5)     17
Name: count, dtype: int64
Intention d'adopter selon ce facteur :
 Series([], Name: intention_adopter, dtype: float64)


KeyError: 'Vous diriez que [Vous avez déjà échangé sur le sujet des véhicules électriques avec votre entourage]_bin'



**Peer Influence: Summary of Results**

**Number of EV users in social circle:**  
- Most respondents report having "Few (1 to 3)" EV users in their social circle (121), followed by "None" (64), "Several (3 to 5)" (30), and "Many (more than 5)" (17).
- There is no clear correlation between the number of EV users in one's social circle and the intention to adopt an EV (correlation coefficient: NaN, likely due to missing or non-numeric data).

**Discussion about EVs with peers:**  
- The majority have "Totally agree" (110) or "Somewhat agree" (82) that they have discussed EVs with their peers.
- Fewer respondents "Somewhat disagree" (28) or "Totally disagree" (12).

**Encouragement from peers:**  
- Most respondents are "Somewhat disagree" (78) or "Somewhat agree" (54) about being encouraged by their peers to adopt an EV.
- Fewer are "Totally agree" (12) or "Totally disagree" (31).

**Perceived positive opinion of peers:**  
- Most respondents "Somewhat agree" (103) or "Somewhat disagree" (93) that their peers have a positive opinion of EVs.
- Fewer "Totally agree" (23) or "Totally disagree" (13).

**Feeling reassured or motivated by peer exchanges:**  
- Most are "Somewhat disagree" (83) or "Somewhat agree" (57) that exchanges with peers reassured or motivated them.
- Fewer "Totally agree" (19) or "Totally disagree" (33).

**Intention to adopt by peer factors:**  
- For all peer influence factors, the mean intention to adopt is not reported (empty series), suggesting either missing data or no significant difference between groups.

**Conclusion:**  
While most respondents have at least some exposure to EV users and discussions in their social circle, there is no strong evidence from this data that peer influence (in terms of number of users, encouragement, or positive opinion) is directly associated with a higher intention to adopt an EV. This may be due to missing data or a lack of strong effect in this sample.

In [ ]:
for col in ["Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique]", "Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet]"]:
    # Mappe les 4 options d'accord/désaccord sur une échelle 0-3
    map4 = {
        "Totalement en désaccord": 0,
        "Plutôt en désaccord": 1,
        "Plutôt d'accord": 2,
        "Totalement d'accord": 3
    }
    df[col + "_bin"] = df[col].map(map4)
    group1 = df[df[col + "_bin"] == 3]["intention_adopter"].dropna()
    group0 = df[df[col + "_bin"] == 0]["intention_adopter"].dropna()
    print(f"{col} - n1: {len(group1)}, n0: {len(group0)}, unique1: {group1.unique()}, unique0: {group0.unique()}")
    print(f"Var group1: {group1.var()}, Var group0: {group0.var()}")
    if len(group1) > 1 and len(group0) > 1:
        tstat, pval = ttest_ind(group1, group0, equal_var=False)
        print(f"{col} : t = {tstat:.2f}, p = {pval:.4f}")
    else:
        print(f"{col} : Pas assez de données pour le t-test.")

Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique] - n1: 7, n0: 8, unique1: [3. 2. 1.], unique0: [1. 3.]
Var group1: 0.6190476190476191, Var group0: 0.5
Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique] : t = 3.03, p = 0.0102
Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet] - n1: 4, n0: 21, unique1: [3. 1.], unique0: [1. 2. 3.]
Var group1: 1.0, Var group0: 0.2285714285714285
Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet] : t = 2.66, p = 0.0700



### **Peer Influence: Statistical Test Results**

#### **Perceived Positive Opinion of Peers**
- **Group sizes:**  
  - "Totally agree" (score 3): n = 7  
  - "Totally disagree" (score 0): n = 8  
- **Adoption intent values (unique):**  
  - "Totally agree": [3.0, 2.0, 1.0]  
  - "Totally disagree": [1.0, 3.0]
- **Variance:**  
  - "Totally agree": 0.62  
  - "Totally disagree": 0.50  
- **t-test:**  
  - t = 3.03, p = 0.0102  
  - **Interpretation:** There is a statistically significant difference in adoption intent between those who totally agree and those who totally disagree that their peers have a positive opinion of EVs.



#### **Feeling Reassured or Motivated by Peer Exchanges**
- **Group sizes:**  
  - "Totally agree" (score 3): n = 4  
  - "Totally disagree" (score 0): n = 21  
- **Adoption intent values (unique):**  
  - "Totally agree": [3.0, 1.0]  
  - "Totally disagree": [1.0, 2.0, 3.0]
- **Variance:**  
  - "Totally agree": 1.00  
  - "Totally disagree": 0.23  
- **t-test:**  
  - t = 2.66, p = 0.0700  
  - **Interpretation:** The difference in adoption intent between these groups is not statistically significant at the 0.05 level, but the result suggests a possible trend.



**Summary:**  
A significantly higher intention to adopt an EV is observed among respondents who perceive their peers as having a positive opinion of EVs. Feeling reassured or motivated by peer exchanges shows a trend toward higher adoption intent, but this result is not statistically significant, possibly due to small group sizes.

## **Clarity and trust in information sources**  
  (Perceived clarity and reliability of information about EVs, Main information sources)

In [ ]:
clarity_col = "Ces sources vous paraissent-elles claires et fiables ?\n"
df["clarity_bin"] = df[clarity_col].map({"Oui, j’ai facilement trouvé ce que je cherchais": 1, "Pas toujours, il faut croiser les infos et c'était plutôt complexe de comprendre les offres et véhicules disponible sur le marché": 0})
group = df.groupby("clarity_bin")["intention_adopter"].mean()
print("Intention d'adopter selon la clarté perçue des sources :\n", group)

Intention d'adopter selon la clarté perçue des sources :
 clarity_bin
0.0    2.433735
1.0    2.263158
Name: intention_adopter, dtype: float64



### **Adoption Intent by Perceived Clarity of Information Sources**

| Perceived Clarity of Sources                | Mean Adoption Intent |
|---------------------------------------------|---------------------|
| Not always clear/complex (0)                | 2.43                |
| Clear and easy to find (1)                  | 2.26                |

- **Interpretation:**  
  Respondents who found information sources "not always clear" or complex reported a slightly higher mean intention to adopt an EV (2.43) compared to those who found sources clear and easy to find (2.26). This difference is small and may not be statistically significant, but it suggests that clarity of information alone does not strongly predict adoption intent in this sample.

In [ ]:
info_col = "Où avez-vous cherché (ou chercheriez-vous) des informations sur les véhicules électriques ?\n"
sources = df[info_col].dropna().str.split(',').explode().str.strip()
source_counts = sources.value_counts()
print("Sources d'information les plus utilisées :\n", source_counts.head(10))

Sources d'information les plus utilisées :
 Où avez-vous cherché (ou chercheriez-vous) des informations sur les véhicules électriques ?\n
Entourage (amis                               125
collègues…)                                   125
Sites des constructeurs / Concessionnaires    111
Médias généralistes / spécialisés             105
Comparateurs de véhicules                      74
Forums / avis en ligne                         64
Réseaux sociaux                                42
Salons ou événements                           23
Non                                             2
YouTube / Influenceurs automobile               2
Name: count, dtype: int64


### **Most Used Information Sources**

| Information Source                        | Count |
|-------------------------------------------|-------|
| Social circle (friends, colleagues, etc.) | 125   |
| Manufacturer/dealer websites              | 111   |
| General/specialized media                 | 105   |
| Vehicle comparison sites                  | 74    |
| Forums / online reviews                   | 64    |
| Social networks                           | 42    |
| Trade shows or events                     | 23    |
| None                                      | 2     |
| YouTube / Automotive influencers          | 2     |

- **Interpretation:**  
  The most frequently cited sources are the respondent’s social circle (friends, colleagues), followed by manufacturer or dealer websites and general or specialized media. Online comparison tools, forums, and social networks are also commonly used, while YouTube/influencers and "none" are rarely mentioned. This highlights the importance of both personal networks and digital media in shaping EV information acquisition.

## **Brand communication**  
  (Brands perceived as offering clear and well-communicated EV options)

In [ ]:
brand_col = "Quelles marques vous semblent proposer une offre électrique claire et bien communiquée ?\n"
brands = df[brand_col].dropna().str.split(',').explode().str.strip()
brand_counts = brands.value_counts()
print("Marques les plus citées :\n", brand_counts.head(10))

Marques les plus citées :
 Quelles marques vous semblent proposer une offre électrique claire et bien communiquée ?\n
Tesla         97
Renault       76
Peugeot       47
BYD           41
Aucune        32
BMW           25
Kia           25
Volkswagen    24
Hyundai       24
Mercedes      20
Name: count, dtype: int64


## **Sociodemographic effects**  
  (Influence of age, professional status, and geographic location on adoption intent)

In [ ]:


# H2 : ANOVA ou t-test selon l'âge, la CSP, la zone géographique
for var in ["Dans quelle tranche d’âge vous situez-vous ?\n", "Habitez-vous dans :", "Quelle est votre situation professionnelle actuelle ?\n"]:
    print(f"\nEffet de {var} sur l'intention d'adopter :")
    groups = [g["intention_adopter"].dropna() for _, g in df.groupby(var) if g["intention_adopter"].notna().sum() > 1]
    if len(groups) >= 2:
        fval, pval = f_oneway(*groups)
        print(f"ANOVA: F = {fval:.2f}, p = {pval:.4f}")
        # Analyse automatique
        if pval < 0.05:
            print("✅ Il existe une différence significative de l'intention d'adopter selon cette variable.")
            means = [g.mean() for g in groups]
            print(f"  Moyennes par groupe : {means}")
        else:
            print("❌ Pas de différence significative de l'intention d'adopter selon cette variable.")
    else:
        print("Pas assez de groupes pour faire une ANOVA valide.")

# H3 : t-test clarté perçue vs intention
group1 = df[df["clarity_bin"] == 1]["intention_adopter"].dropna()
group0 = df[df["clarity_bin"] == 0]["intention_adopter"].dropna()
tstat, pval = ttest_ind(group1, group0, equal_var=False)
print(f"H3 - Clarté perçue : t = {tstat:.2f}, p = {pval:.4f}")
# Analyse automatique
if pval < 0.05:
    print("✅ L'intention d'adopter est significativement plus élevée chez ceux qui trouvent les sources claires et fiables.")
    print(f"  Moyenne (sources claires) : {group1.mean():.2f}")
    print(f"  Moyenne (sources peu claires) : {group0.mean():.2f}")
else:
    print("❌ Pas de différence significative d'intention d'adopter selon la clarté perçue des sources.")


Effet de Dans quelle tranche d’âge vous situez-vous ?
 sur l'intention d'adopter :
ANOVA: F = 1.64, p = 0.1396
❌ Pas de différence significative de l'intention d'adopter selon cette variable.

Effet de Habitez-vous dans : sur l'intention d'adopter :
ANOVA: F = 3.53, p = 0.0167
✅ Il existe une différence significative de l'intention d'adopter selon cette variable.
  Moyennes par groupe : [np.float64(2.4464285714285716), np.float64(2.2142857142857144), np.float64(1.84), np.float64(2.473684210526316)]

Effet de Quelle est votre situation professionnelle actuelle ?
 sur l'intention d'adopter :
ANOVA: F = 0.82, p = 0.5176
❌ Pas de différence significative de l'intention d'adopter selon cette variable.
H3 - Clarté perçue : t = -0.74, p = 0.4658
❌ Pas de différence significative d'intention d'adopter selon la clarté perçue des sources.


 **Sociodemographic Effects on Adoption Intent**

| Variable                                 | Test   | Statistic (F/t) | p-value  | Significant? | Group Means (if significant)                |
|-------------------------------------------|--------|-----------------|----------|--------------|---------------------------------------------|
| Age group                                | ANOVA  | F = 1.64        | 0.1396   | ❌ No         | —                                           |
| Geographic location                      | ANOVA  | F = 3.53        | 0.0167   | ✅ Yes        | 2.45, 2.21, 1.84, 2.47                      |
| Professional status                      | ANOVA  | F = 0.82        | 0.5176   | ❌ No         | —                                           |
| Perceived clarity of information sources | t-test | t = -0.74       | 0.4658   | ❌ No         | —                                           |

- **Interpretation:**  
  - Only geographic location shows a statistically significant effect on intention to adopt an EV (p = 0.0167), with mean adoption intent varying by region.
  - Age group, professional status, and perceived clarity of information sources do **not** show significant effects on adoption intent in this sample.

## **multivariate (cross) analyses** 